# Exome processing: project LA FE
<pre>
metadata {
    date:
    folder:
    operator:
    scripts:
    
}
</pre>

<pre>
pmg@SRVBIOL01:[/DATA/workspace/projects/0040_exomes_projects--production/0040.4-LaFE/analysis/0040.4-analysis-2017-07-19]
[devel|GRCh37|ens-75|python3.6venv1]$ find ../../fastq/  -name '*fastq.gz' | grep -vi undeter > lafe_fastq.fof  
</pre>

In [176]:
import sys
import os
import re
import pprint
import pandas as pd
from natsort import natsorted, index_natsorted, order_by_index

pp = pprint.PrettyPrinter(indent=4)
pp = pp.pprint

print("Pandas version: {}".format(pd.__version__))

# get data and current dir
cwd = os.getcwd()
print ( 'current dir is: "' + cwd + '"')

Pandas version: 0.20.2
current dir is: "/DATA/workspace/projects/0040_exomes_projects--production/0040.4-LaFE/analysis/0040.4-analysis-2017-07-19"


###  natural human numerical sort
<div style="border:2px solid black;">
<pre>
#https://nedbatchelder.com/blog/200712/human_sorting.html
import re

def tryint(s):
    try:
        return int(s)
    except:
        return s
     
def alphanum_key(s):
    """ Turn a string into a list of string and number chunks.
        "z23a" -> ["z", 23, "a"]
    """
    return [ tryint(c) for c in re.split('([0-9]+)', s) ]

def sort_nicely(l):
    """ Sort the given list in the way that humans expect.
    """
    l.sort(key=alphanum_key)
</pre>
</div>

<div style="border:2px solid black;">
<pre>
# https://stackoverflow.com/questions/5967500/how-to-correctly-sort-a-string-with-a-number-inside
import re

def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    '''
    # https://stackoverflow.com/questions/5967500/how-to-correctly-sort-a-string-with-a-number-inside
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    '''
    return [ atoi(c) for c in re.split('(\d+)', text) ]
</pre>
</div>

In [150]:
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    '''
    # https://stackoverflow.com/questions/5967500/how-to-correctly-sort-a-string-with-a-number-inside
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    '''
    return [ atoi(c) for c in re.split('(\d+)', text) ]

def sort_human(elements):
    """ Sort the given list in the way that humans expect.
    """
    elements.sort(key=natural_keys)

In [273]:
##
self = {
    "config" : {
        "project" : "LaFe-exome_agilent",
        "fastqs_path":"/DATA/workspace/projects/0040_exomes_projects--production/0040.4-LaFE/fastq",
        "analysis_folder_path" : "/DATA/workspace/projects/0040_exomes_projects--production/0040.4-LaFE/analysis/0040.4-analysis-2017-07-19",
        "bams_path" : '',
        "vcfs_path" : '',
        "QC_path" : '',
        "fastq_fof": 'lafe_fastq.fof',
    },
    'data':{
        'sample_list':None,
        'fastq_df': None,
        'ref_b37': '/DATA/references_and_genomic_db_devel/genodb/hs37d5_1kg/hs37d5.fa.gz',
        'align_cmd_by_sample':{},
    },
    'workflow' : {
        'steps' : get_steps()
    }
}

In [247]:
%%bash
ls /DATA/workspace/projects/0040_exomes_projects--production/0040.4-LaFE/analysis/0040.4-analysis-2017-07-19/

Agilent_exome-LAFE_r1--alignmnet.ipynb
exome_LAFE-2017-07-20--preprocess_fastq.ipynb
lafe_fastq.fof
README
test_self.p


## Pipeline Steps

In [274]:
# STEPS

def get_steps() :
    
    steps = {
        "Step_1" : {"cmd": '',
                    "desc": '',
                    "label" : 'check folder and dirs',
                    "depends_on": None
                  },
        "Step_2" : {"cmd": '',
                    "desc": '',
                    "label" : 'fastq_qc',
                    "depends_on" : 'Step_1'
                 },
        "Step_3" : {"cmd": '',
                    "desc": '',
                    "label" : 'bwa_mem',
                    "depends_on" : 'Step_1'
                  },
        "Step_3.1" : {"cmd": '',
                    "desc": '',
                    "label" : 'baqm_rg',
                    "depends_on" : 'Step_3'
                  },
        "Step_3.2" : {"cmd": '',
                    "desc": '',
                    "label" : 'mark_duplicates',
                    "depends_on" : 'Step_3.1'
                  },
        "Step_3.3" : {"cmd": '',
                    "desc": '',
                    "label" : 'rmdup',
                    "depends_on" : 'Step_3.1'
                  },
        "Step_3qc" : {"cmd": '',
                    "desc": '',
                    "label" : 'bwa_qc',
                     "depends_on" : 'Step_3'
                  },
        "Step_4" : {"cmd": '',
                    "desc": '',
                    "label" : 'coverage',
                    "depends_on" : 'Step_3'
                  },
        "Step_4qc" : {"cmd": '',
                    "desc": '',
                    "label" : 'coverage_qc',
                    "depends_on" : 'Step_4'
                  },
        "Step_5" : {"cmd": '',
                   "desc": '',
                   "label" : 'vcf_generation',
                    "depends_on" : 'Step_3'
                  },
        "Step_5qc" : {"cmd": '',
                   "desc": '',
                   "label" : 'vcf_qc',
                     "depends_on" : 'Step_5'
                  },
        "Step_6" : {"cmd": '',
                   "desc": '',
                   "label" : 'annotation',
                    "depends_on" : 'Step_5'
                  },
        "Step_6qc" : {"cmd": '',
                      "desc": '',
                      "label" : 'annotation_qc',
                      "depends_on" : 'Step_6'
                  }
    }
    return steps

def show_steps(action=None):
    """
    Function to show the steps to be followed by the pipeline
    Input: action -> 'by_label'
    Output: Shows the steps to the user in the computer screen
    """
    steps = get_steps()
    
    if action == 'by_label':
        #dict = {x: {'label':steps[x]['label']} for x in steps}
        #pp(dict)
        for x,y  in  get_steps().items():
            print (x + ': ' + y['label'])
    else:
          pp(steps)
    

In [275]:
show_steps('by_label')

Step_1: check folder and dirs
Step_2: fastq_qc
Step_3: bwa_mem
Step_3qc: bwa_qc
Step_4: coverage
Step_4qc: coverage_qc
Step_5: vcf_generation
Step_5qc: vcf_qc
Step_6: annotation
Step_6qc: annotation_qc


## Fastq file list

In [276]:
fastq_files = []
config_dat = self['config']
fastq_fof = os.path.join(config_dat['analysis_folder_path'], config_dat['fastq_fof'])

print("reading file '{}'".format(fastq_fof))

reading file '/DATA/workspace/projects/0040_exomes_projects--production/0040.4-LaFE/analysis/0040.4-analysis-2017-07-19/lafe_fastq.fof'


In [277]:
with open(fastq_fof) as f:
    fastq_files = f.read().splitlines()
    
for x in fastq_files:
    print(x)

../../fastq/E176_LAFE/Exoma_E176_La_Fe-42696655/RPN-318-51454483/RPN-318_S1_L001_R1_001.fastq.gz
../../fastq/E176_LAFE/Exoma_E176_La_Fe-42696655/RPN-318-51454483/RPN-318_S1_L002_R1_001.fastq.gz
../../fastq/E176_LAFE/Exoma_E176_La_Fe-42696655/RPN-318-51454483/RPN-318_S1_L004_R1_001.fastq.gz
../../fastq/E176_LAFE/Exoma_E176_La_Fe-42696655/RPN-318-51454483/RPN-318_S1_L001_R2_001.fastq.gz
../../fastq/E176_LAFE/Exoma_E176_La_Fe-42696655/RPN-318-51454483/RPN-318_S1_L002_R2_001.fastq.gz
../../fastq/E176_LAFE/Exoma_E176_La_Fe-42696655/RPN-318-51454483/RPN-318_S1_L003_R2_001.fastq.gz
../../fastq/E176_LAFE/Exoma_E176_La_Fe-42696655/RPN-318-51454483/RPN-318_S1_L003_R1_001.fastq.gz
../../fastq/E176_LAFE/Exoma_E176_La_Fe-42696655/RPN-318-51454483/RPN-318_S1_L004_R2_001.fastq.gz
../../fastq/E176_LAFE/Exoma_E176_La_Fe-42696655/RPN-315-51465437/RPN-315_S6_L002_R1_001.fastq.gz
../../fastq/E176_LAFE/Exoma_E176_La_Fe-42696655/RPN-315-51465437/RPN-315_S6_L004_R1_001.fastq.gz
../../fastq/E176_LAFE/Exoma_E1

## samples

Get df: exome, sample

capture patters: 
    * exome  /\/Exoma_(E_\d+)/
    * sample /\/(RPN-\d+)/

In [278]:
# construct a dataframe from fastq path names
fastq_AoA = []
for x in fastq_files:
    print(x)
    sample = re.search("/Exoma_(E\d+).+?/(RPN-\d+).+/(RPN-\d+.+?fastq.gz)",x)
    if sample:
        row=list(sample.groups())
        row.append(x)
        print(row)
        fastq_AoA.append(row)
    else:
        print("** NO SAMPLE**")
        
pp(fastq_AoA)

../../fastq/E176_LAFE/Exoma_E176_La_Fe-42696655/RPN-318-51454483/RPN-318_S1_L001_R1_001.fastq.gz
['E176', 'RPN-318', 'RPN-318_S1_L001_R1_001.fastq.gz', '../../fastq/E176_LAFE/Exoma_E176_La_Fe-42696655/RPN-318-51454483/RPN-318_S1_L001_R1_001.fastq.gz']
../../fastq/E176_LAFE/Exoma_E176_La_Fe-42696655/RPN-318-51454483/RPN-318_S1_L002_R1_001.fastq.gz
['E176', 'RPN-318', 'RPN-318_S1_L002_R1_001.fastq.gz', '../../fastq/E176_LAFE/Exoma_E176_La_Fe-42696655/RPN-318-51454483/RPN-318_S1_L002_R1_001.fastq.gz']
../../fastq/E176_LAFE/Exoma_E176_La_Fe-42696655/RPN-318-51454483/RPN-318_S1_L004_R1_001.fastq.gz
['E176', 'RPN-318', 'RPN-318_S1_L004_R1_001.fastq.gz', '../../fastq/E176_LAFE/Exoma_E176_La_Fe-42696655/RPN-318-51454483/RPN-318_S1_L004_R1_001.fastq.gz']
../../fastq/E176_LAFE/Exoma_E176_La_Fe-42696655/RPN-318-51454483/RPN-318_S1_L001_R2_001.fastq.gz
['E176', 'RPN-318', 'RPN-318_S1_L001_R2_001.fastq.gz', '../../fastq/E176_LAFE/Exoma_E176_La_Fe-42696655/RPN-318-51454483/RPN-318_S1_L001_R2_001.fas

In [279]:
# create the Pandas dataframe
df_fastq = pd.DataFrame(fastq_AoA, columns =["assay", "sample", "fastq_file", "fastq_path"])
## <TO_DO> pandas sorting with external function for natural_human_sorting
df_fastq.sort_values(by=["fastq_file"], ascending=[True])

,assay,sample,fastq_file,fastq_path
82,E177,RPN-1059,RPN-1059_S2_L001_R1_001.fastq.gz,../../fastq/E177_LAFE/Exoma_E177_LaFe-42766724...
83,E177,RPN-1059,RPN-1059_S2_L001_R2_001.fastq.gz,../../fastq/E177_LAFE/Exoma_E177_LaFe-42766724...
85,E177,RPN-1059,RPN-1059_S2_L002_R1_001.fastq.gz,../../fastq/E177_LAFE/Exoma_E177_LaFe-42766724...
84,E177,RPN-1059,RPN-1059_S2_L002_R2_001.fastq.gz,../../fastq/E177_LAFE/Exoma_E177_LaFe-42766724...
81,E177,RPN-1059,RPN-1059_S2_L003_R1_001.fastq.gz,../../fastq/E177_LAFE/Exoma_E177_LaFe-42766724...
86,E177,RPN-1059,RPN-1059_S2_L003_R2_001.fastq.gz,../../fastq/E177_LAFE/Exoma_E177_LaFe-42766724...
80,E177,RPN-1059,RPN-1059_S2_L004_R1_001.fastq.gz,../../fastq/E177_LAFE/Exoma_E177_LaFe-42766724...
87,E177,RPN-1059,RPN-1059_S2_L004_R2_001.fastq.gz,../../fastq/E177_LAFE/Exoma_E177_LaFe-42766724...
75,E177,RPN-1278,RPN-1278_S7_L001_R1_001.fastq.gz,../../fastq/E177_LAFE/Exoma_E177_LaFe-42766724...
79,E177,RPN-1278,RPN-1278_S7_L001_R2_001.fastq.gz,../../fastq/E177_LAFE/Exoma_E177_LaFe-42766724...


In [280]:
df_fastq.columns


Index(['assay', 'sample', 'fastq_file', 'fastq_path'], dtype='object')

In [281]:
# find number of fastq by samples
df_fastq['sample'].value_counts()

RPN-295     8
RPN-2017    8
RPN-284     8
RPN-357     8
RPN-387     8
RPN-318     8
RPN-293     8
RPN-311     8
RPN-1636    8
RPN-1612    8
RPN-1059    8
RPN-2027    8
RPN-315     8
RPN-1278    8
Name: sample, dtype: int64

### all samples OK
<pre>
RPN-295     8
RPN-2017    8
RPN-284     8
RPN-357     8
RPN-387     8
RPN-318     8
RPN-293     8
RPN-311     8
RPN-1636    8
RPN-1612    8
RPN-1059    8
RPN-2027    8
RPN-315     8
RPN-1278    8
Name: sample, dtype: int64
</pre>

### add samples to self

In [292]:
#self['data']['sample_list'] = 


# get samples sorted with sort_human
samples = list(df_fastq['sample'].unique())
sort_human(samples)
pp(samples)
#self['data']['sample_list'] = samples
#self['data']['sample_list']

## store
with open('samples.txt','w') as ofh:
    ofh.write('\n'.join(samples)+"\n")

[   'RPN-284',
    'RPN-293',
    'RPN-295',
    'RPN-311',
    'RPN-315',
    'RPN-318',
    'RPN-357',
    'RPN-387',
    'RPN-1059',
    'RPN-1278',
    'RPN-1612',
    'RPN-1636',
    'RPN-2017',
    'RPN-2027']


### Process sample by sample

In [283]:
df_fastq[df_fastq['sample']=='RPN-284'].sort_values(['fastq_file'])

,assay,sample,fastq_file,fastq_path
53,E176,RPN-284,RPN-284_S5_L001_R1_001.fastq.gz,../../fastq/E176_LAFE/Exoma_E176_La_Fe-4269665...
51,E176,RPN-284,RPN-284_S5_L001_R2_001.fastq.gz,../../fastq/E176_LAFE/Exoma_E176_La_Fe-4269665...
50,E176,RPN-284,RPN-284_S5_L002_R1_001.fastq.gz,../../fastq/E176_LAFE/Exoma_E176_La_Fe-4269665...
54,E176,RPN-284,RPN-284_S5_L002_R2_001.fastq.gz,../../fastq/E176_LAFE/Exoma_E176_La_Fe-4269665...
48,E176,RPN-284,RPN-284_S5_L003_R1_001.fastq.gz,../../fastq/E176_LAFE/Exoma_E176_La_Fe-4269665...
49,E176,RPN-284,RPN-284_S5_L003_R2_001.fastq.gz,../../fastq/E176_LAFE/Exoma_E176_La_Fe-4269665...
55,E176,RPN-284,RPN-284_S5_L004_R1_001.fastq.gz,../../fastq/E176_LAFE/Exoma_E176_La_Fe-4269665...
52,E176,RPN-284,RPN-284_S5_L004_R2_001.fastq.gz,../../fastq/E176_LAFE/Exoma_E176_La_Fe-4269665...


In [284]:
df_fastq_sorted = df_fastq.reindex(index=order_by_index(df_fastq.index, index_natsorted(df_fastq.fastq_file)))

In [285]:
print(df_fastq_sorted['sample'].unique())
len(df_fastq_sorted['sample'].unique())

['RPN-284' 'RPN-293' 'RPN-295' 'RPN-311' 'RPN-315' 'RPN-318' 'RPN-357'
 'RPN-387' 'RPN-1059' 'RPN-1278' 'RPN-1612' 'RPN-1636' 'RPN-2017'
 'RPN-2027']


14

In [286]:
# get samples sorted in dataframe
self['data']['sample_list'] = list(df_fastq_sorted['sample'].unique())
self['data']['fastq_df'] = df_fastq_sorted

In [287]:
self

{'config': {'QC_path': '',
  'analysis_folder_path': '/DATA/workspace/projects/0040_exomes_projects--production/0040.4-LaFE/analysis/0040.4-analysis-2017-07-19',
  'bams_path': '',
  'fastq_fof': 'lafe_fastq.fof',
  'fastqs_path': '/DATA/workspace/projects/0040_exomes_projects--production/0040.4-LaFE/fastq',
  'project': 'LaFe-exome_agilent',
  'vcfs_path': ''},
 'data': {'align_cmd_by_sample': {},
  'fastq_df':     assay    sample                        fastq_file  \
  53   E176   RPN-284   RPN-284_S5_L001_R1_001.fastq.gz   
  51   E176   RPN-284   RPN-284_S5_L001_R2_001.fastq.gz   
  50   E176   RPN-284   RPN-284_S5_L002_R1_001.fastq.gz   
  54   E176   RPN-284   RPN-284_S5_L002_R2_001.fastq.gz   
  48   E176   RPN-284   RPN-284_S5_L003_R1_001.fastq.gz   
  49   E176   RPN-284   RPN-284_S5_L003_R2_001.fastq.gz   
  55   E176   RPN-284   RPN-284_S5_L004_R1_001.fastq.gz   
  52   E176   RPN-284   RPN-284_S5_L004_R2_001.fastq.gz   
  33   E176   RPN-293   RPN-293_S7_L001_R1_001.fastq.gz

In [288]:
import pickle
pickle.dump(self, open('test_self.p', 'wb'))